# CSCI 4022 Final Project 
By John Danekind and Daniel Hatakeyama

## Import Stuff


In [2]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn 
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import pyminhash

# Apparently we have to implement our own algos from scratch which is gay af. I just have this here for now

### Ideal questions to consider for now
- Can unsupervised clustering naturally separate fake from real news without using labels?
- What distinctive linguistic patterns emerge in clusters of fake vs. real news?
- Are there identifiable sub-categories within fake news that clustering can reveal?
- How do temporal patterns differ between fake and real news clusters?
- Which textual features most strongly contribute to the separation of clusters?


#### Things to think about
- Dataset is meant for more supervised learning tasks (Neural nets, svms, knn, etc)
- Maybe use KMeans, minhashing, and GMM's to cluster news into different categories
- Compare these clusters to the labels later 


#### Current plan since we have no time lmao
- Make datasets into one big dataset with no labels.
- Do K means on that data and cluster articles by similarity (could be jaccard, euclidean with vector embeddings, something else (figure out later))
- See what patterns arise from this. 
- Then take the regular data set and do a simple supervised method that is interpretable. (Logistic regression, random forest. NO BLACK BOX)
- Compare how each of the methods performed. 
- Can fake vs real news be clustered without explicit labels or is it harder to detect?

## Read in the data as DataFrames for now 

In [8]:
# Load the data sets
fake_df = pd.read_csv('../data/Fake.csv')
fake_df['label'] = 0
real_df = pd.read_csv('../data/True.csv')
real_df['label'] = 1

fake_df.head()





,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [9]:
real_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [16]:
# Combine the data sets
df = pd.concat([fake_df, real_df], ignore_index=True)
df = df.dropna()
df = df.drop_duplicates()

# Combine title and text 
df['content'] = df['title'] + '. ' + df['text']

df.head() 

,title,text,subject,date,label,content
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0,Donald Trump Sends Out Embarrassing New Year’...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0,Drunk Bragging Trump Staffer Started Russian ...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0,Sheriff David Clarke Becomes An Internet Joke...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0,Trump Is So Obsessed He Even Has Obama’s Name...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0,Pope Francis Just Called Out Donald Trump Dur...


#### Claude doing claude things (fix this shit later)

#### Text Processing pipeline (Look into more later )

In [17]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = str(text).lower()
    
    # Remove special characters, numbers, punctuation
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    
    # Tokenize
    tokens = text.split()
    
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    # Rejoin tokens
    return ' '.join(tokens)

# Apply preprocessing
df['processed_content'] = df['content'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /Users/john/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /Users/john/nltk_data...


In [18]:
df.head(5)

,title,text,subject,date,label,content,processed_content
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0,Donald Trump Sends Out Embarrassing New Year’...,donald trump sends embarrassing new year eve m...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0,Drunk Bragging Trump Staffer Started Russian ...,drunk bragging trump staffer started russian c...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0,Sheriff David Clarke Becomes An Internet Joke...,sheriff david clarke becomes internet joke thr...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0,Trump Is So Obsessed He Even Has Obama’s Name...,trump obsessed even obama name coded website i...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0,Pope Francis Just Called Out Donald Trump Dur...,pope francis called donald trump christmas spe...


### Feature Extraction (Figure out how it works later)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000, min_df=5, max_df=0.85)

# Fit and transform the processed content
X = vectorizer.fit_transform(df['processed_content'])
feature_names = vectorizer.get_feature_names_out()
y = df['label']  # Your labels are already 0 and 1

print(f"TF-IDF matrix shape: {X.shape}")

TF-IDF matrix shape: (44689, 5000)
